In [2]:
from fyenn_class import  pd_loaddata, pd, np 
path = './data_down/'
os.listdir(path)


['huawei_daily.csv', 'huawei_output.csv', '入湖数据关系.xlsx']

In [80]:
%%time
huawei_output = pd_loaddata.pd_sep001(path + 'huawei_output.csv')
# huawei_daily = pd_loaddata.pd_sep001(path + 'huawei_daily.csv')
rel = pd_loaddata.pd_excel(path + '入湖数据关系', 2)
huawei_output.shape

Wall time: 222 ms


(11717, 33)

In [81]:
pd_loaddata.show_line(15, None)
rel = rel.drop(['部门', '站点'],axis = 1).drop_duplicates();rel

,序号,对应现有看板业务量指标,对应效率指标,OU（成本中心）
0,1,按条收货-原材料,inbound,HUAWEDHW4S
3,2,按条发货-原材料,outbound,HUAWEDHW4S
23,3,PSN-原材料,labelling,HUAWEDHW4S
24,4,按条收货-荣耀,inbound,HONORDGHMS
26,5,按条发货-荣耀,outbound,HONORDGHMS
36,6,PSN-荣耀,labelling,HONORDGHMS
37,7,珠三角仓配趟数,transport,HUAWEDHWTS
39,8,贵阳仓配趟数,transport,HUAWEDGTRD
41,9,成品收货-T园区,inbound,HUAWEDGLSS
42,10,成品发货-T园区,outbound,HUAWEDGLSS


In [83]:
rel.iloc[:, 3].value_counts()

HUAWEDHW4S    3
HONORDGHMS    3
NEXPEDGIHS    3
HUAWEDGLSS    2
HUAWEDHWTS    1
HUAWEDGTRD    1
HUAWEDHW1S    1
HUAWEDGNHS    1
Name: OU（成本中心）, dtype: int64

In [84]:
def datetime_(coach, col):
    coach[col] = coach[col].astype(str).str.slice(0,10)
    coach = pd.concat([coach, pd.DataFrame(list(coach[col].str.split('-')))], axis =1)
    coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
    coach = coach.rename({0:'year', 1:'month', 2: 'date'}, axis=1)
    return coach
huawei_output = datetime_(coach=huawei_output, col='update_date').drop(['createtime', 'updatetime'], axis = 1)

def search_col(df, str):
    return list(pd.Series(df.columns)[pd.Series(df.columns).str.match(str)])

In [265]:
tran_col = list(pd.Series(huawei_output.columns)[
    pd.Series(huawei_output.columns).str.contains(
        '(receive|psn|send|transport_times|update_date|year|month|date|id)'
        ) == False
        ])

tran_col
# clean , case, withdraw)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_24992/2230356562.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  pd.Series(huawei_output.columns).str.contains(


['t_product_clean', 'anshi_case', 'pre_withdraw']

In [274]:
re = 't_'
pd.Series(tran_col).str.extract("(" + re + ".+)")[0][0]

't_product_clean'

In [277]:
def concat_(re, ou):
    huawei_output[search_col(huawei_output, re)].shape
    # n = 4 - huawei_output[search_col(huawei_output, re)].shape[1]
    # m = huawei_output[search_col(huawei_output, re)].shape[0]
    # print(m, n)
    data = pd.concat(
        [
            huawei_output[search_col(huawei_output, re)], 
            # pd.DataFrame(np.zeros(shape=(m, n), dtype=int)),
            huawei_output[['update_date', 'year', 'month', 'date']]
        ]
        , axis = 1).sort_values('update_date')
    # print(data.shape)
    data.columns = list(pd.Series(data.columns).str.extract(
        '(receive|psn|send|transport_times|update_date|year|month|date)'
        )[0])
    try:
        data = data.rename({np.nan:'addition'}, axis = 1)
    except:
        pass
    data['ou'] = ou
    data['addition_type'] = pd.Series(tran_col).str.extract("(" + re + ".+)")[0][0]

    return data.reset_index(drop=True)

In [281]:
relist = ['hon', 'origin', 'pearlriver_transport_times', 'guiyang_transport_times', 't_', 'r4_', 'nanh', 'anshi', 'te', 'pre']
oulist = list(rel['OU（成本中心）'].unique())

In [251]:
origin = concat_('origin', 'HUAWEDHW4S')
honor = concat_('hon', 'HONORDGHMS') 

In [280]:
list(rel['OU（成本中心）'].unique())

['HUAWEDHW4S',
 'HONORDGHMS',
 'HUAWEDHWTS',
 'HUAWEDGTRD',
 'HUAWEDGLSS',
 'HUAWEDHW1S',
 'HUAWEDGNHS',
 'NEXPEDGIHS']

In [278]:
rel

,序号,对应现有看板业务量指标,对应效率指标,OU（成本中心）
0,1,按条收货-原材料,inbound,HUAWEDHW4S
3,2,按条发货-原材料,outbound,HUAWEDHW4S
23,3,PSN-原材料,labelling,HUAWEDHW4S
24,4,按条收货-荣耀,inbound,HONORDGHMS
26,5,按条发货-荣耀,outbound,HONORDGHMS
36,6,PSN-荣耀,labelling,HONORDGHMS
37,7,珠三角仓配趟数,transport,HUAWEDHWTS
39,8,贵阳仓配趟数,transport,HUAWEDGTRD
41,9,成品收货-T园区,inbound,HUAWEDGLSS
42,10,成品发货-T园区,outbound,HUAWEDGLSS


In [240]:
dict(zip(tran_col, np.repeat('additional',3)))

{'t_product_clean': 'additional',
 'anshi_case': 'additional',
 'pre_withdraw': 'additional'}

In [254]:
huawei_output.columns

Index(['update_date', 'origin_receive', 'origin_send', 'origin_psn',
       'honor_receive', 'honer_send', 'honor_psn',
       'pearlriver_transport_times', 'guiyang_transport_times',
       't_product_receive', 't_product_clean', 't_product_send', 'r4_receive',
       'r4_send', 'r4_psn', 'nanhua_receive', 'nanhua_send', 'nanhau_psn',
       'anshi_receive', 'anshi_send', 'anshi_case', 'te_origin_receive',
       'te_origin_send', 'te_product_receive', 'te_product_send',
       'pre_receive', 'pre_withdraw', 'honor_transport_times', 'id', 'appid',
       'entryid', 'year', 'month', 'date'],
      dtype='object')